In [2]:
from transformers import BartForConditionalGeneration, AutoTokenizer
bart = BartForConditionalGeneration.from_pretrained('hyunwoongko/kobart')
tokenizer = AutoTokenizer.from_pretrained('hyunwoongko/kobart')

In [3]:
texts = ['최근 들어 딥러닝', '안녕 친구야']
sample_ids = tokenizer(texts, padding=True, return_tensors='pt')['input_ids']
sample_ids

tensor([[19261, 14301,  1700, 10021, 10277,  9747],
        [11699,  9592, 15230, 11734,     3,     3]])

In [4]:
outs = bart.generate(
    sample_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    early_stopping=True,
)

In [31]:
import torch
import torch.nn.functional as F

def rerank(matrix, repeat_time, margin=0.):
    upper = matrix.unsqueeze(-1).repeat_interleave(repeat_time, dim=-1)
    lower = matrix.unsqueeze(1).repeat_interleave(repeat_time, dim=1)
    return F.relu((lower - upper + margin).triu(1))

In [32]:
batch_size = 8
num_cands = 16
# # embed_size = docs['input_ids'].size(-1)
# assert num_cands == cands['input_ids'].size(0) // batch_size

# doc_embeddings = self.encoder(**docs)[0][:, 0, :]
# gold_embeddings = self.encoder(**golds)[0][:, 0, :]
# sims_doc_gold = torch.cosine_similarity(doc_embeddings, gold_embeddings, dim=-1)

# cand_embeddings = self.encoder(**cands)[0][:, 0, :].view(batch_size, num_cands, -1)
# doc_embeddings = doc_embeddings.unsqueeze(1).repeat_interleave(num_cands, dim=1)
# sims_doc_cand = torch.cosine_similarity(doc_embeddings, cand_embeddings, dim=-1)

# scores_gold = sims_doc_cand - sims_doc_gold.unsqueeze(1).repeat_interleave(num_cands, dim=1)
# scores_gold = F.relu(scores_gold).sum(1)

ranks = (
    torch.arange(num_cands)
    .unsqueeze(0)
    .repeat_interleave(batch_size, dim=0)
)
rerank(ranks, num_cands)

tensor([[[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.],
         [ 0.,  0.,  0.,  ..., 11., 12., 13.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  1.,  2.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  1.,  2.,  ..., 13., 14., 15.],
         [ 0.,  0.,  1.,  ..., 12., 13., 14.]

In [33]:
import pandas as pd
pd.DataFrame({'ab': [[1,2,3],[1,2,3]]})

,ab
0,"[1, 2, 3]"
1,"[1, 2, 3]"


In [34]:
from datasets import load_metric
rouge = load_metric('rouge')

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [47]:
predictions = ['abc ccc', 'edf']
references = ['abc ccc', 'edf']
# ['최근 들어 딥러닝은 음성, 시각, 자연어, 로봇, 게임 등 인공지능 여러 분야의', '환경을 근본적으로 변화시키고 있다.']
# references = ['최근 들어 딥러닝은 음성, 시각, 자연어, 로봇, 게임 등 인공지능 여러 분야의', '환경을 근본적으로 변화시키고 있다.']
scores = rouge.compute(predictions=predictions, references=references[::-1])

In [48]:
scores['rouge1'].mid.fmeasure

0.0

In [1]:
valid_path = '/Volumes/GoogleDrive/My Drive/gassum/data/valid_w_cands.csv'

In [2]:
import pandas as pd

In [3]:
valid = pd.read_csv(valid_path)

In [5]:
valid = valid.iloc[:, 2:]

In [15]:
eval(valid['candidates'][1])

['바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있는 것으로 알려졌다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하여 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.',
 '바른미래당이 8일 연 최고위원 회의에는 7명의 최고위원 중 손학규 대표와 김관영 원내대표만 참석하였고 바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧하는 등, 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.']